In [1]:
%load_ext autoreload
%autoreload 2
import gym
import os
import numpy as np
import random
import torch
from torch import nn
from torch.utils.tensorboard import SummaryWriter
from experiment import rollout, ReplayBuffer, Trajectory, load_model, save_model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
MODEL_NAME = 'model_v0_lunar_lander_v2'
HIDDEN = 64
curr_step = 0
writer = SummaryWriter()

class Behavior(torch.nn.Module):
    def __init__(self, input_shape, num_actions):
        super(Behavior, self).__init__()
        self.classifier = torch.nn.Sequential(
            nn.Linear(input_shape, HIDDEN), 
            nn.ReLU(),
            nn.Linear(HIDDEN, HIDDEN),      
            nn.ReLU(),            
            nn.Linear(HIDDEN, HIDDEN),      
            nn.ReLU(),            
            nn.Linear(HIDDEN, HIDDEN),       
            nn.ReLU(),
            nn.Linear(HIDDEN, num_actions)
        )        

    def forward(self, x):
        return self.classifier(x)

In [3]:
env = gym.make('LunarLander-v2')

In [4]:
loss_object = torch.nn.CrossEntropyLoss().to(device)
model_sample = Behavior(input_shape=env.observation_space.shape[0]+2, num_actions=env.action_space.n).to(device)
optimizer = torch.optim.Adam(model_sample.parameters())

In [5]:
rb = ReplayBuffer(max_size=500, last_few=25)

n_warmup_episodes = 10
# Random rollout
trajectories, avg_reward, length = rollout(episodes=n_warmup_episodes, env=env, render=False)
rb.add(trajectories)


# Keep track of steps used during random rollout!
epoch, model_sample, optimizer, loss, steps = load_model(MODEL_NAME, model_sample, optimizer, device, train=True)
steps += length
save_model(MODEL_NAME, epoch, model_sample, optimizer, loss, steps)
writer.add_scalar('Mean_Reward', avg_reward, steps)


print(f"Average Episode Reward: {avg_reward}")

No checkpoint found. Creating new model.
Average Episode Reward: -196.17139496307186


In [6]:
batch_size = 1024

In [7]:
def train_step(model, inputs, targets):
    optimizer.zero_grad()    
    predictions = model(inputs)
    loss = loss_object(predictions, targets)
    
    loss.backward()
    optimizer.step()
    
    return loss

def action_fn(model, inputs, sample_action=True):
    action_logits = model(inputs)
    action_probs = torch.softmax(action_logits, axis=-1)

    if sample_action:        
        m = torch.distributions.categorical.Categorical(logits=action_logits)             
        action = int(m.sample().squeeze().cpu().numpy())        
    else:
        action = int(np.argmax(action_probs.detach().squeeze().numpy()))
    return action
    

In [8]:
# SAMPLE ACTIONS

loss_sum = 0
loss_count = 0

epochs = 1000000
epoch, model_sample, optimizer, loss, steps = load_model(MODEL_NAME, model_sample, optimizer, device, train=True)
print(steps)


for i in range(epoch, epochs+epoch):
    x, y = rb.sample(batch_size, device)    
    loss = train_step(model_sample, x, y)
    loss_sum += loss
    loss_count += 1
    
    writer.add_scalar('Loss/train', loss, i)
    
    n_episodes_per_iter = 10
    n_updates_per_iter = 300
    if i % n_updates_per_iter == 0:        
        trajectories, mean_reward, length = rollout(n_episodes_per_iter, env=env, model=model_sample, sample_action=True, replay_buffer=rb, 
                              device=device, action_fn=action_fn)
        rb.add(trajectories)
        
        steps += length
        avg_loss = loss_sum/loss_count
        save_model(MODEL_NAME, i, model_sample, optimizer, avg_loss, steps)        
        print(f"Average Episode Reward: {mean_reward}")        
        writer.add_scalar('Mean_Reward', mean_reward, steps)
        

    if i % 200 == 0:
        avg_loss = loss_sum/loss_count
        print(f'i: {i}, s: {steps}, Loss: {avg_loss}')
        
        save_model(MODEL_NAME, i, model_sample, optimizer, avg_loss, steps)

Existing model found. Loading from epoch 0, steps 873 with loss: 0.0
873
Average Episode Reward: -270.8382872397604
i: 0, s: 1762, Loss: 1.4075428247451782
Average Episode Reward: -479.4280424597431
i: 200, s: 2814, Loss: 1.3656699657440186
Average Episode Reward: -259.0408612220241
i: 400, s: 3693, Loss: 1.3508198261260986
Average Episode Reward: -326.2165240048614
i: 600, s: 4692, Loss: 1.3410078287124634
Average Episode Reward: -281.82230039545027
i: 800, s: 5614, Loss: 1.332353115081787
Average Episode Reward: -305.13289893327845
i: 1000, s: 6544, Loss: 1.3239821195602417
Average Episode Reward: -291.1179316894763
i: 1200, s: 7445, Loss: 1.316978096961975
Average Episode Reward: -296.30426978792366
i: 1400, s: 8311, Loss: 1.310514211654663
Average Episode Reward: -277.79088870576777
i: 1600, s: 9384, Loss: 1.3048697710037231
Average Episode Reward: -309.97205954199114
i: 1800, s: 10349, Loss: 1.2992591857910156
Average Episode Reward: -229.6182587336529
i: 2000, s: 11219, Loss: 1.2

KeyboardInterrupt: 

In [ ]:
rb.sample_command()

In [ ]:
cmd = (280.085, 318.91295076177306)
rb.sample_command()
env = gym.make('LunarLander-v2')
e, model, _, l = load_model(name=MODEL_NAME, train=False, model=model_sample, optimizer=optimizer, device=device)

_, mean_reward = rollout(episodes=5, env=env, model=model_sample, sample_action=True, 
                      cmd=cmd, render=True, device=device, action_fn=action_fn)


print(f"Average Episode Reward: {mean_reward}")